In [1]:
from jetbot import Camera, bgr8_to_jpeg
import ipywidgets.widgets as widgets
import traitlets
import numpy as np
import cv2

# 初始化相机
camera = Camera.instance(width=720, height=720)

# 创建显示控件
image_widget = widgets.Image(format='jpeg', width=360, height=360)
mask_widget = widgets.Image(format='jpeg', width=360, height=360)
result_widget = widgets.Image(format='jpeg', width=360, height=360)

# 创建滑块控件
h_low = widgets.IntSlider(min=0, max=180, value=0, description='H min')
h_high = widgets.IntSlider(min=0, max=180, value=10, description='H max')
s_low = widgets.IntSlider(min=0, max=255, value=43, description='S min')
s_high = widgets.IntSlider(min=0, max=255, value=255, description='S max')
v_low = widgets.IntSlider(min=0, max=255, value=46, description='V min')
v_high = widgets.IntSlider(min=0, max=255, value=255, description='V max')

# 布局
controls = widgets.VBox([h_low, h_high, s_low, s_high, v_low, v_high])
images = widgets.HBox([image_widget, mask_widget, result_widget])
ui = widgets.VBox([controls, images])

def update_image(change):
    # 获取当前帧
    frame = camera.value
    
    # 转换为HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # 获取当前阈值
    lower = np.array([h_low.value, s_low.value, v_low.value])
    upper = np.array([h_high.value, s_high.value, v_high.value])
    
    # 创建掩膜
    mask = cv2.inRange(hsv, lower, upper)
    
    # 应用掩膜
    result = cv2.bitwise_and(frame, frame, mask=mask)
    
    # 显示图像
    image_widget.value = bgr8_to_jpeg(frame)
    mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR))
    result_widget.value = bgr8_to_jpeg(result)

# 绑定滑块事件
for slider in [h_low, h_high, s_low, s_high, v_low, v_high]:
    slider.observe(update_image, names='value')

# 初始更新
update_image(None)

# 显示UI
display(ui)